In [1]:
from utils import *
from normalizers import *
from FLPO import *
from viz import *
from UAV_Net import UAV_Net
from annealing import anneal
import pickle
import time

In [2]:
import random
import numpy as np

# fix all the seeds for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
drones = [
    ((10.0, 5.0), (45.0, 50.0), 1.0),  # Long distance, high charge
    ((3.0, 40.0), (50.0, 10.0), 1.0),  # Long distance, medium charge
    ((20.0, 15.0), (35.0, 35.0), 1.0),  # Moderate distance, medium charge
    ((5.0, 30.0), (25.0, 5.0), 1.0),  # Moderate distance, low charge
    ((40.0, 45.0), (10.0, 10.0), 1.0),  # Long distance, high charge
    ((30.0, 20.0), (5.0, 35.0), 1.0),  # Moderate distance, medium charge
    ((15.0, 10.0), (40.0, 40.0), 1.0),  # Moderate distance, low charge
    ((35.0, 5.0), (10.0, 45.0), 1.0),  # Long distance, medium charge
    ((25.0, 40.0), (20.0, 10.0), 1.0),  # Moderate distance, high charge
    ((45.0, 15.0), (5.0, 20.0), 1.0),  # Long distance, low charge
]
N_stations = 5
init_ugv = np.repeat(
    np.array(
        [
            [0.5, 0.9],
        ]
    ),
    N_stations,
    axis=0,
)
fcr = 15.0  # Full Charge Range
ugv_factor = 0.0  # the cost factor for UGV transportation
distance = "euclidean"  # distance measure in the environment
blocks = None # for benchmark we ignore blocks


n_algo_iters = 1
scale = 47.0 # normalizing dataset into a unit box
# FOR BENCHMARK ALGORITHMS #########################
s = np.array([ [list(start)] for start, _, _ in drones ]) / scale
e = np.array([ [list(end)] for _, end, _ in drones ]) / scale
num_nodes = N_stations
num_agents = len(drones)
T = num_nodes + 1
dim_ = 2
F_base = init_ugv.copy()
threshold = fcr / scale

# FLPO Algorithm

In [3]:
beta_init = 1e-4  # initial beta value for the optimization.
beta_f = 1e4  # final beta value for the optimization
alpha = 2.0  # beta growth rate
purturb = 1e-6  # random purturbation in optimization
t_flpo_arr = []
c_flpo_arr = []
for i in range(n_algo_iters):
    start_time = time.time()
    uav_net = UAV_Net(drones, N_stations, init_ugv, blocks, ugv_factor, fcr, distance)
    obj = uav_net.objective
    Y_s, Betas = anneal(
        obj,
        uav_net.stations,
        uav_net.bounds,
        beta_init,
        beta_f,
        alpha,
        purturb,
        "powell",
        False,
    )
    t_flpo_arr.append(time.time() - start_time)
    c_flpo_arr.append(uav_net.return_total_cost())

UAV Network was successfully created.
--Optimization Terminated--
Elapsed time: 11.53


# GUROBI

In [4]:
from Benchmark.GurobiSolver import SolveMIP

gap = 0.0
t_gurobi_arr = []
c_gurobi_arr = []
for i in range(n_algo_iters):
    best_y, best_eta, cost, elapsed_time = SolveMIP(
        s,
        e,
        num_agents,
        num_nodes,
        T,
        dim_,
        threshold=threshold,
        y_init=None,
        gap=gap,
    )
    t_gurobi_arr.append(elapsed_time)
    c_gurobi_arr.append(cost * scale * num_agents)

Set parameter Username
Academic license - for non-commercial use only - expires 2026-07-28
Read LP format model from file C:\Users\salar\AppData\Local\Temp\tmplbn5d751.pyomo.lp
Reading time = 0.00 seconds
x1: 228 rows, 550 columns, 844 nonzeros
Set parameter NonConvex to value 2
Set parameter MIPGap to value 0
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13620H, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 228 rows, 550 columns and 844 nonzeros
Model fingerprint: 0x0f1ab7e6
Model has 180 quadratic constraints
Variable types: 190 continuous, 360 integer (360 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e-01, 1e+00]
  Objective range  [1e-01, 1e-01]
  Bounds range     [5e-02, 1e+00]
  RHS range        [3e-01, 1e+00]
  QRHS range       [6e-02, 1e+0

# GA

In [5]:
from Benchmark.GA import ga, ga_plot, ga_print

t_ga_arr = []
c_ga_arr = []
for i in range(n_algo_iters):
    best_y, best_eta, best_cost, elapsed_time = ga(
        s,
        e,
        num_nodes,
        num_agents,
        dim_,
        threshold=threshold,
        Y_init=None,
        verbose=False,
        pop_size=100,
        generations=10, # 10000
        mutation_rate=0.3,
    )
    t_ga_arr.append(elapsed_time)
    c_ga_arr.append(best_cost * scale)

# SA

In [6]:
from Benchmark.SA import sa, plot_sa, print_sa
t_sa_arr = []
c_sa_arr = []

for i in range(n_algo_iters):
    best_y, best_eta, best_cost, elapsed_time = sa(
            s,
            e,
            num_nodes,
            num_agents,
            dim_,
            threshold=threshold,
            Y_init=None,
            YMIN=0,
            YMAX=1.0,
            T_start=10,
            iters=5, # 50000
            verbose=False,
        )
    t_sa_arr.append(elapsed_time)
    c_sa_arr.append(best_cost * scale)


# CEM

In [7]:
from Benchmark.CEM import cem, plot_cem, print_cem

t_cem_arr = []
c_cem_arr = []
for i in range(n_algo_iters):
    best_y, best_eta, best_cost, elapsed_time = cem(
            s,
            e,
            num_nodes,
            num_agents,
            dim_,
            threshold=threshold,
            Y_init=None,
            YMIN=0,
            YMAX=1.0,
            n_iter=2,# 2000
            pop_size=200,
            elite_frac=0.1,
            verbose=False,
        )
    t_cem_arr.append(elapsed_time)
    c_cem_arr.append(best_cost * scale)


# PSO

In [8]:
from Benchmark.PSO import pso, pso_plot, pso_print

num_particles = 500
num_iters = 2 # 2000
w = 0.9 # inertia weight
c1 = 0.5  # cognitive (individual) weight
c2 =1.5  # social (group) weight
t_pso_arr = []
c_pso_arr = []
for i in range(n_algo_iters):
    y_final, eta_final, global_best_cost, elapsed_time = pso(
        s,
        e,
        num_nodes,
        num_agents,
        dim_,
        threshold= threshold,
        Y_init=None,
        verbose=False,
        w=w,
        c1=c1,
        c2=c2,
        num_particles=num_particles,
        num_iters=num_iters,
    )
    t_pso_arr.append(elapsed_time)
    c_pso_arr.append(global_best_cost * scale)



In [9]:
import pickle

benchmark_data = {
    "s":s, 
    "e":e,
    "n_algo_iters":n_algo_iters,
    "t_pso_arr":t_pso_arr,
    "c_pso_arr":c_pso_arr,
    "t_ga_arr":t_ga_arr,
    "c_ga_arr":c_ga_arr,
    "t_sa_arr":t_sa_arr,
    "c_sa_arr":c_sa_arr,
    "t_cem_arr":t_cem_arr,
    "c_cem_arr":c_cem_arr,
    "t_flpoANN_arr":t_flpo_arr,
    "c_flpoANN_arr":c_flpo_arr,
    "t_gurobi_arr":t_gurobi_arr,
    "c_gurobi_arr":c_gurobi_arr
}

scenario = f"N{int(num_agents)}_M{int(num_nodes)}_seed{int(seed)}"
filepath = "Benchmark/" + scenario
print(filepath)

with open(filepath, 'wb') as file:
    pickle.dump(benchmark_data, file)

Benchmark/N10_M5_seed42


In [10]:
# extract min, mean, and std for each algorithm (cost and time)
def extract_stats(arr):
    if arr is None:
        return None, None, None
    return np.min(arr), np.mean(arr), np.std(arr)
t_pso_min, t_pso_mean, t_pso_std = extract_stats(t_pso_arr)
c_pso_min, c_pso_mean, c_pso_std = extract_stats(c_pso_arr)
t_ga_min, t_ga_mean, t_ga_std = extract_stats(t_ga_arr)
c_ga_min, c_ga_mean, c_ga_std = extract_stats(c_ga_arr)
t_sa_min, t_sa_mean, t_sa_std = extract_stats(t_sa_arr)
c_sa_min, c_sa_mean, c_sa_std = extract_stats(c_sa_arr)
t_cem_min, t_cem_mean, t_cem_std = extract_stats(t_cem_arr)
c_cem_min, c_cem_mean, c_cem_std = extract_stats(c_cem_arr)
t_flpoANN_min, t_flpoANN_mean, t_flpoANN_std = extract_stats(t_flpo_arr)
c_flpoANN_min, c_flpoANN_mean, c_flpoANN_std = extract_stats(c_flpo_arr)
t_gurobi_min, t_gurobi_mean, t_gurobi_std = extract_stats(t_gurobi_arr)
c_gurobi_min, c_gurobi_mean, c_gurobi_std = extract_stats(c_gurobi_arr)
# print the results
# print(f"PSO: time (min, mean, std) = ({t_pso_min:.5f}, {t_pso_mean:.5f}, {t_pso_std:.5f}), cost (min, mean, std) = ({c_pso_min:.5f}, {c_pso_mean:.5f}, {c_pso_std:.5f})")
print(f"GA: time (min, mean, std) = ({t_ga_min:.5f}, {t_ga_mean:.5f}, {t_ga_std:.5f}), cost (min, mean, std) = ({c_ga_min:.5f}, {c_ga_mean:.5f}, {c_ga_std:.5f})")
print(f"SA: time (min, mean, std) = ({t_sa_min:.5f}, {t_sa_mean:.5f}, {t_sa_std:.5f}), cost (min, mean, std) = ({c_sa_min:.5f}, {c_sa_mean:.5f}, {c_sa_std:.5f})")
print(f"CEM: time (min, mean, std) = ({t_cem_min:.5f}, {t_cem_mean:.5f}, {t_cem_std:.5f}), cost (min, mean, std) = ({c_cem_min:.5f}, {c_cem_mean:.5f}, {c_cem_std:.5f})")
# print(f"ACO: time (min, mean, std) = ({t_aco_min:.5f}, {t_aco_mean:.5f}, {t_aco_std:.5f}), cost (min, mean, std) = ({c_aco_min:.5f}, {c_aco_mean:.5f}, {c_aco_std:.5f})")
print(f"FLPO: time (min, mean, std) = ({t_flpoANN_min:.5f}, {t_flpoANN_mean:.5f}, {t_flpoANN_std:.5f}), cost (min, mean, std) = ({c_flpoANN_min:.5f}, {c_flpoANN_mean:.5f}, {c_flpoANN_std:.5f})")
# now for gurobi
print(f"Gurobi: time (min, mean, std) = ({t_gurobi_min:.5f}, {t_gurobi_mean:.5f}, {t_gurobi_std:.5f}), cost (min, mean, std) = ({c_gurobi_min:.5f}, {c_gurobi_mean:.5f}, {c_gurobi_std:.5f})")


GA: time (min, mean, std) = (0.66259, 0.66259, 0.00000), cost (min, mean, std) = (45737.59570, 45737.59570, 0.00000)
SA: time (min, mean, std) = (0.00451, 0.00451, 0.00000), cost (min, mean, std) = (125978.20004, 125978.20004, 0.00000)
CEM: time (min, mean, std) = (0.51392, 0.51392, 0.00000), cost (min, mean, std) = (109622.92445, 109622.92445, 0.00000)
FLPO: time (min, mean, std) = (11.52704, 11.52704, 0.00000), cost (min, mean, std) = (446.42547, 446.42547, 0.00000)
Gurobi: time (min, mean, std) = (36.54043, 36.54043, 0.00000), cost (min, mean, std) = (434.08871, 434.08871, 0.00000)


In [11]:
import pandas as pd

# Prepare data for the table with std in parentheses next to mean, all values in two decimals
def fmt(mean, std):
    return f"{mean:.2f} ({std:.2f})"

data = {
    "Algorithm": ["PSO", "GA", "SA", "CEM", "FLPO", "Gurobi"],
    "Cost Min": [f"{c_pso_min:.2f}", f"{c_ga_min:.2f}", f"{c_sa_min:.2f}", f"{c_cem_min:.2f}", f"{c_flpoANN_min:.2f}", f"{c_gurobi_min:.2f}"],
    "Cost Mean (Std)": [
        fmt(c_pso_mean, c_pso_std),
        fmt(c_ga_mean, c_ga_std),
        fmt(c_sa_mean, c_sa_std),
        fmt(c_cem_mean, c_cem_std),
        fmt(c_flpoANN_mean, c_flpoANN_std),
        fmt(c_gurobi_mean, c_gurobi_std),
    ],
    "Time Min": [f"{t_pso_min:.2f}", f"{t_ga_min:.2f}", f"{t_sa_min:.2f}", f"{t_cem_min:.2f}", f"{t_flpoANN_min:.2f}", f"{t_gurobi_min:.2f}"],
    "Time Mean (Std)": [
        fmt(t_pso_mean, t_pso_std),
        fmt(t_ga_mean, t_ga_std),
        fmt(t_sa_mean, t_sa_std),
        fmt(t_cem_mean, t_cem_std),
        fmt(t_flpoANN_mean, t_flpoANN_std),
        fmt(t_gurobi_mean, t_gurobi_std),
    ],
}

df_benchmark = pd.DataFrame(data)
print(df_benchmark)

  Algorithm   Cost Min   Cost Mean (Std) Time Min Time Mean (Std)
0       PSO   39313.16   39313.16 (0.00)     0.84     0.84 (0.00)
1        GA   45737.60   45737.60 (0.00)     0.66     0.66 (0.00)
2        SA  125978.20  125978.20 (0.00)     0.00     0.00 (0.00)
3       CEM  109622.92  109622.92 (0.00)     0.51     0.51 (0.00)
4      FLPO     446.43     446.43 (0.00)    11.53    11.53 (0.00)
5    Gurobi     434.09     434.09 (0.00)    36.54    36.54 (0.00)
